In [2]:
import numpy as np

In [1]:
def LB_preparation(self):
    """
    Run to prepare a dictionary of orthognoal basis LB, for which the key is a tuple (i,j,k)
    Then it will be stored in self.LB
    """
    m_max,tmin,tmax,keys = np.max(self.m),self.theta_min,self.theta_max,self.keyboard[m_max]
    B_w = self.Legendre_B(tmin[0],tmax[0],m_max)
    B_a = self.Legendre_B(tmin[1],tmax[1],m_max)    
    B_b = self.Legendre_B(tmin[2],tmax[2],m_max)
    LB = {}
    for key in keys:
        i,j,k = key
        LB[key] = lambda theta,i=i,j=j,k=k:B_w[i](theta[0])*B_a[j](theta[1])*B_b[k](theta[2])
    self.LB = LB

In [3]:
def basic_interation_cal(self,r_pre,s_pre,theta):
    """
    r_pre,s_pre of dim = d
    
    Return:
            r,s : dim = d
    """
    ones = np.eye(self.n)
    D_t_pre = np.diag(np.reshape(s_pre))
    R_t_pre = np.diag(np.reshape(r_pre))
    
    D_t_2 = ones*theta[0] + theta[1]*R_t_pre**2 + theta[2]*D_t_pre**2
    D_t = np.sqrt(D_t_2)
    V_t = D_t.dot(self.GAMMA).dot(D_t)
    r_t = np.random.multivariate_normal(np.zeros(self.n), V_t)
    return r_t,np.diag(D_t)

In [4]:
def theta_generator(self,n=1):
    the = np.array([self.theta_max[i] - self.theta_min[i] for i in range(3)])*np.random.uniform(size = (n,3)) + np.array(self.theta_min)
    return np.reshape(the,3) if n==1 else the

In [5]:
def simple_generation(self,r,s):
    """
    input r,s: series of dim=d, (r and sigma)
    ----------
    output: new r,s
    """
    the = self.theta_generator()
    return basic_interation_cal(r,s,the)

In [6]:
def r_preparetion(self):
    """
    with r0,sigma0, we will be able to calculate all r_t with t<=T
    """
    r,s = self.r_init,self.sigma_init
    r_t = [r]
    s_t = [s]
    for t in range(self.T):
        r,s = self.simple_generation(r,s)
        r_t.append(r)
        s_t.append(s)
    self.r_t = r_t
    self.s_t = s_t